<a href="https://colab.research.google.com/github/GHes31415/Research_OT/blob/main/grad_desc_wass_bary_gauss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import time
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Here I'm going to try the followin loss funciton for finding the barycenter of Gaussian distributions in the case $n = 2$


In [57]:
n = 10
sampling_cov_wish = torch.distributions.wishart.Wishart(n,covariance_matrix=torch.eye(n))
sampling_cov_lkj = torch.distributions.lkj_cholesky.LKJCholesky(n,0.5)
# sig_1 = sampling_cov.sample()
# sig_2 = torch.tensor([[3,1],[1,5]],dtype= torch.float32)#.to(device)
# sig_3 = torch.tensor([[5,-4],[-4,6]],dtype= torch.float32)
# sig_4 = torch.tensor([[15,-8],[-8,60]],dtype= torch.float32)
# sig_5 = torch.tensor([[0.1,0.001],[0.01,1]],dtype= torch.float32)
# sig_6 = torch.tensor([[10000,0.001],[0.01,1]],dtype= torch.float32)
# sig_7 = torch.tensor([[10000,-0.001],[-0.01,1]],dtype= torch.float32)
# cov_mats = [sig_1,sig_2,sig_3,sig_4,sig_5,sig_6,sig_7]
k = 9
cov_mats_wish = []
cov_mats_lkj = []
for i in range(k):
  L = sampling_cov_lkj.sample()
  cov_mats_lkj.append((i+1)*L@L.T)
  # L2 = sampling_cov_wish.sample()
  # cov_mats_wish.append(L2@L2.T)

# print(cov_mats_lkj)

# print(cov_mats_wish)

In [40]:
# Check for commutativity
# torch.norm(sig_1@sig_2-sig_2@sig_1)

In [41]:
def sqrt_cov(sig: torch.tensor, inverse= False):
  '''
  sig: nxn SPD matrix
  '''
  D,U = torch.linalg.eig(sig)
  if inverse:
    return torch.matmul(torch.matmul(U,torch.diag(D**0.5)),torch.linalg.inv(U)).type(torch.float32),torch.matmul(torch.matmul(U,torch.diag(D**(-0.5))),torch.linalg.inv(U)).type(torch.float32)
  else:
    return torch.matmul(torch.matmul(U,torch.diag(D**0.5)),torch.linalg.inv(U)).type(torch.float32)

In [42]:
def loss(theta,cov_mats,k):
  '''
  theta:    torch.tensor size n^2,
            current estimate.
  cov_mats: list of size k of tensors with shape nxn,
            covariance matrices.
  k:        torch.tensor size 1,
            number of covariance matrices.
  '''
  theta_m = theta.reshape((n,n))#.to(device)
  sig_theta = torch.matmul(theta_m,theta_m.T)
  sum_loss = (wass_distance(sig_theta,cov_mats,k)).sum()



  return sum_loss/k

def loss2(theta,cov_mats,k):
  #Maybe this has to be modified to be squared
  sum_loss = (dis_par_to_covs(theta,cov_mats,k)).sum()
  return sum_loss/k
def loss3(theta,cov_mats,k):
  #Maybe this has to be modified to be squared
  sum_loss = (dis_par_to_covs_2(theta,cov_mats,k)).sum()
  return sum_loss/k

In [43]:
def dis_par_to_covs(theta,cov_mats,k):
  '''
  theta:    torch.tensor size n^2,
            current estimate.
  cov_mats: list of size k of tensors with shape nxn,
            covariance matrices.
  k:        torch.tensor size 1,
            number of covariance matrices.
  '''
  theta_m = theta.reshape((n,n))#.to(device)
  sig_theta = torch.matmul(theta_m,theta_m.T)
  sig_theta_sqrt,sig_theta_sqrt_inv = sqrt_cov(sig_theta,True)
  # sig_theta_sqrt = sqrt_cov(sig_theta)
  # sig_theta_sqrt_inv = torch.linalg.inv(sig_theta_sqrt)
  distances = torch.zeros(k)
  for i,sig in enumerate(cov_mats):
    first_mat = torch.matmul(torch.matmul(sig_theta_sqrt,sig),sig_theta_sqrt)
    sqrt_mat = sqrt_cov(first_mat)
    second_mat = torch.matmul(sig_theta_sqrt_inv,sqrt_mat)
    distances[i] =torch.linalg.matrix_norm(sig_theta_sqrt-second_mat)**2
  return distances
def dis_par_to_covs_2(theta,cov_mats,k):
  '''
  theta:    torch.tensor size n^2,
            current estimate.
  cov_mats: list of size k of tensors with shape nxn,
            covariance matrices.
  k:        torch.tensor size 1,
            number of covariance matrices.
  '''
  theta_m = theta.reshape((n,n))#.to(device)
  sig_theta = torch.matmul(theta_m,theta_m.T)
  distances = torch.zeros(k)
  for i,sig in enumerate(cov_mats):
    sig_sqrt,sig_sqrt_inv = sqrt_cov(sig,True)
    first_mat = sqrt_cov(torch.matmul(torch.matmul(sig_sqrt,sig_theta),sig_sqrt))
    second_mat = torch.matmul(sig_sqrt_inv,first_mat)
    distances[i] =torch.linalg.matrix_norm(sig_sqrt-second_mat)**2
  return distances
def dis_cov_to_covs(cov,cov_mats,k):
  cov_sqrt,cov_sqrt_inv = sqrt_cov(cov,True)
  distances = torch.zeros(k)
  for i,sig in enumerate(cov_mats):
    first_mat = torch.matmul(torch.matmul(cov_sqrt,sig),cov_sqrt)
    # print(first_mat)
    sqrt_mat = sqrt_cov(first_mat)
    second_mat = torch.matmul(cov_sqrt_inv,sqrt_mat)
    distances[i] = torch.linalg.matrix_norm(cov_sqrt-second_mat)**2



  return distances

In [44]:
def wass_distance(cov,cov_mats,k):
  cov_sqrt = sqrt_cov(cov)
  distances = torch.zeros(k)
  for i,sig in enumerate(cov_mats):
    mat = sqrt_cov(torch.matmul(torch.matmul(cov_sqrt,sig),cov_sqrt))
    distances[i] = torch.trace(cov+sig-2*mat)

  return distances


In [45]:

# theta = torch.randn(n**2,requires_grad=True)#.to(device)
# optimizer = torch.optim.Adam([theta],lr = lr)

In [46]:

def training(theta,lr,n_iters,loss,cov_mats):
  increment = 1
  for epoch in range(n_iters):

    # compute loss and backpropagate
    l = loss(theta,cov_mats,k)
    l.backward()
    d_theta = theta.grad
    # update theta gradient descent
    with torch.no_grad():
      theta -= lr*d_theta
    if torch.norm(lr*d_theta)<10**(-6):
      print(epoch,torch.norm(lr*d_theta))
      return theta
    if epoch%1000 == 0:
      print(epoch,torch.norm(lr*d_theta))
    # zero graidents
    theta.grad.zero_()
  return theta







In [74]:
lr = torch.tensor(.01)#.to(device)
n_iters = 5000

In [75]:
t1 = time.time()
theta = torch.randn(n**2,requires_grad=True)#.to(device)
theta_w2 = training(theta,lr,n_iters,loss,cov_mats_lkj)
print(time.time()-t1)

0 tensor(0.0984)
1000 tensor(1.1242e-06)
1032 tensor(9.9649e-07)
10.367677450180054


In [76]:
t1 = time.time()
theta = torch.randn(n**2,requires_grad=True)#.to(device)
theta_par = training(theta,lr,n_iters,loss2,cov_mats_lkj)
print(time.time()-t1)

0 tensor(0.1440)
940 tensor(9.8008e-07)
9.180063962936401


In [77]:
t1 = time.time()
theta = torch.randn(n**2,requires_grad=True)#.to(device)
theta_par = training(theta,lr,n_iters,loss3,cov_mats_lkj)
print(time.time()-t1)

0 tensor(0.1079)
1000 tensor(3.5540e-05)
2000 tensor(3.5623e-05)
3000 tensor(4.1888e-05)
4000 tensor(4.6841e-05)
62.14758515357971


In [50]:
def fixed_point_it_sol(n,cov_mat,k,n_iters):
  sig = torch.randn(n,n)
  sig = torch.matmul(sig,sig.T)
  sig_sqrt,sig_sqrt_inv = sqrt_cov(sig,True)
  increase = 1
  # counter = 0
  for epoch in range(n_iters):
    # print(sig)

    sum_mats = torch.zeros_like(sig)
    for cov in cov_mat:
      sum_mats += sqrt_cov(torch.matmul(torch.matmul(sig_sqrt,cov),sig_sqrt))/k
    mat = torch.matmul(sum_mats,sum_mats)
    sig_next = torch.matmul(torch.matmul(sig_sqrt_inv,mat),sig_sqrt_inv)
    # print(f'sig next{sig_next}')
    increase = torch.linalg.matrix_norm(sig-sig_next)
    sig = sig_next.clone()
    sig_sqrt,sig_sqrt_inv = sqrt_cov(sig,True)
    if increase<10**(-6):
      print(epoch,increase)
      return sig
    if epoch%1000==0:
      print(epoch,increase)
  print(epoch)
  return sig

In [66]:
t1 = time.time()
sig = fixed_point_it_sol(n,cov_mats_lkj,k,n_iters)
print(time.time()-t1)

0 tensor(50.2833)
1000 tensor(3.7318e-05)
2000 tensor(3.0690e-05)
3000 tensor(2.4552e-05)
4000 tensor(1.8431e-05)
4999
14.427969217300415
